In [ ]:
from config import API_TOKEN
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import pandas as pd

url = "https://hibernate.atlassian.net/rest/api/3/search"

username = "omar.khabthani123@gmail.com"
api_token = API_TOKEN
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}
auth = (username, api_token)

jql = "project = 'HHH'"

params = {
    "jql": jql,
    "maxResults": 1000  
}
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
response = requests.get(url, headers=headers, params=params, auth=auth,verify=False)

if response.status_code == 200:
    data = response.json()

    issues = [{
        "Key": issue["key"],
        "Summary": issue["fields"]["summary"],
        "Status": issue["fields"]["status"]["name"],
        "Assignee": issue["fields"]["assignee"]["displayName"] if issue["fields"]["assignee"] else None,
        "Created": issue["fields"]["created"],
        "Reporter": issue["fields"]["reporter"]["displayName"] if issue["fields"]["reporter"] else None,
        "Resolution": issue["fields"]["resolution"]["name"] if issue["fields"]["resolution"] else None,
        "Priority": issue["fields"]["priority"]["name"] if issue["fields"]["priority"] else None,
        "Updated": issue["fields"]["updated"],
        "Type": issue["fields"]["issuetype"]["name"],
        "Affects Version": issue["fields"]["versions"][0]["name"] if issue["fields"]["versions"] else None,
        "Components": [component["name"] for component in issue["fields"]["components"]],
        "Fix Versions": [version["name"] for version in issue["fields"]["fixVersions"]],
        #"Worked In": issue["fields"]["customfield_11700"],  # Replace XXXXX with the custom field ID for "Worked In"
        "Status Category": issue["fields"]["status"]["statusCategory"]["name"]
        
    } for issue in data["issues"]]

    df = pd.DataFrame(issues)
    excel_filename = "jira_fields.xlsx"
    df.to_excel(excel_filename, index=False)
    print(f"DataFrame exported to {excel_filename} successfully.")

else:
    print("Error:", response.status_code, response.text)

DataFrame exported to jira_fields.xlsx successfully.


In [ ]:
#extracts description field
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import json
from bs4 import BeautifulSoup
import pandas as pd

url = "https://hibernate.atlassian.net/rest/api/3/search"

username = "omar.khabthani123@gmail.com"
api_token = APIKEY
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}
auth = (username, api_token)

jql = "project = 'HHH'"

params = {
    "jql": jql,
    "maxResults": 1000,  
    "expand": "renderedFields"  
}
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
response = requests.get(url, headers=headers, params=params, auth=auth, verify=False)

if response.status_code == 200:
    data = response.json()

    issues = [{
        "Key": issue["key"],
        "Summary": issue["fields"]["summary"],
        "Status": issue["fields"]["status"]["name"],
        "Assignee": issue["fields"]["assignee"]["displayName"] if issue["fields"]["assignee"] else None,
        "Created": issue["fields"]["created"],
        "Reporter": issue["fields"]["reporter"]["displayName"] if issue["fields"]["reporter"] else None,
        "Resolution": issue["fields"]["resolution"]["name"] if issue["fields"]["resolution"] else None,
        "Priority": issue["fields"]["priority"]["name"] if issue["fields"]["priority"] else None,
        "Updated": issue["fields"]["updated"],
        "Type": issue["fields"]["issuetype"]["name"],
        "Affects Version": issue["fields"]["versions"][0]["name"] if issue["fields"]["versions"] else None,
        "Components": [component["name"] for component in issue["fields"]["components"]],
        "Fix Versions": [version["name"] for version in issue["fields"]["fixVersions"]],
        #"Worked In": issue["fields"]["customfield_11700"],  # Replace XXXXX with the custom field ID for "Worked In"
        "Status Category": issue["fields"]["status"]["statusCategory"]["name"],
        "Description": BeautifulSoup(issue["renderedFields"]["description"], 'html.parser').get_text() if "renderedFields" in issue else None,
        
    } for issue in data["issues"]]

    df = pd.DataFrame(issues)
    excel_filename = "jira_fields.xlsx"
    df.to_excel(excel_filename, index=False)
    print(f"DataFrame exported to {excel_filename} successfully.")
    
else:
    print("Error:", response.status_code, response.text)

DataFrame exported to jira_fields.xlsx successfully.


In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

def fetch_all_project_keys():
    url = "https://hibernate.atlassian.net/rest/api/3/project"
    auth = ("omar.khabthani123@gmail.com",APIKEY)
    headers = {"Accept": "application/json"}
    response = requests.get(url, headers=headers, auth=auth)
    response.raise_for_status()
    data = response.json()
    return [project["key"] for project in data]

def fetch_issue_data(project_key):
    url = "https://hibernate.atlassian.net/rest/api/3/search"
    auth = ("omar.khabthani123@gmail.com",APIKEY)
    headers = {"Accept": "application/json", "Content-Type": "application/json"}
    params = {
        "jql": f"project={project_key}",
        "maxResults": 1000,  
        "expand": "renderedFields"  
    }
    response = requests.get(url, headers=headers, params=params, auth=auth)
    response.raise_for_status()
    data = response.json()

    issues = []
    for issue in data["issues"]:
        description = None
        if "renderedFields" in issue and "description" in issue["renderedFields"]:
            description = BeautifulSoup(issue["renderedFields"]["description"], 'html.parser').get_text()
        
        affects_version = None
        if "versions" in issue["fields"] and issue["fields"]["versions"]:
            affects_version = issue["fields"]["versions"][0]["name"]
        fix_versions = None
        if "fixVersions" in issue["fields"] and issue["fields"]["fixVersions"]:
            fix_versions = [version["name"] for version in issue["fields"]["fixVersions"]]
        
        issues.append({
            "Project": project_key,
            "Key": issue["key"],
            "Summary": issue["fields"]["summary"],
            "Status": issue["fields"]["status"]["name"],
            "Assignee": issue["fields"]["assignee"]["displayName"] if issue["fields"]["assignee"] else None,
            "Created": issue["fields"]["created"],
            "Reporter": issue["fields"]["reporter"]["displayName"] if issue["fields"]["reporter"] else None,
            "Resolution": issue["fields"]["resolution"]["name"] if issue["fields"]["resolution"] else None,
            "Priority": issue["fields"]["priority"]["name"] if issue["fields"]["priority"] else None,
            "Updated": issue["fields"]["updated"],
            "Type": issue["fields"]["issuetype"]["name"],
            "Affects Version": affects_version,
            "Components": [component["name"] for component in issue["fields"]["components"]],
            "Fix Versions": fix_versions,
            "Status Category": issue["fields"]["status"]["statusCategory"]["name"],
            "Description": description,
        })

    return issues

if __name__ == "__main__":
    username = "omar.khabthani123@gmail.com"
    api_token = APIKEY
    
    all_project_keys = fetch_all_project_keys()
    all_issues = []

    for project_key in all_project_keys:
        project_issues = fetch_issue_data(project_key)
        all_issues.extend(project_issues)
    
    df = pd.DataFrame(all_issues)
    excel_filename = "jira_fields.xlsx"
    df.to_excel(excel_filename, index=False)
    print(f"DataFrame exported to {excel_filename} successfully.")

DataFrame exported to jira_fields.xlsx successfully.


In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import re

def clean_text(text):
    cleaned_text = re.sub(r'[^\x00-\x7F]+', '', text)
    cleaned_text = re.sub(r'[\000-\010]|[\013-\014]|[\016-\037]', '', cleaned_text)
    return cleaned_text

def fetch_all_project_keys():
    url = "https://hibernate.atlassian.net/rest/api/3/project"
    auth = ("omar.khabthani123@gmail.com",APIKEY)
    headers = {"Accept": "application/json"}
    response = requests.get(url, headers=headers, auth=auth)
    response.raise_for_status()
    data = response.json()
    return [project["key"] for project in data]

def fetch_issue_data(project_key):
    url = "https://hibernate.atlassian.net/rest/api/3/search"
    auth = ("omar.khabthani123@gmail.com",APIKEY)
    headers = {"Accept": "application/json", "Content-Type": "application/json"}
    
    all_issues = []
    start_at = 0
    page_size = 50  
    
    while True:
        params = {
            "jql": f"project={project_key}",
            "startAt": start_at,
            "maxResults": page_size,
            "expand": "renderedFields"  
        }
        response = requests.get(url, headers=headers, params=params, auth=auth)
        response.raise_for_status()
        data = response.json()

        for issue in data["issues"]:
            description = None
            if "renderedFields" in issue and "description" in issue["renderedFields"]:
                description = BeautifulSoup(issue["renderedFields"]["description"], 'html.parser').get_text()
                description = clean_text(description)
            
            affects_version = None
            if "versions" in issue["fields"] and issue["fields"]["versions"]:
                affects_version = issue["fields"]["versions"][0]["name"]
            fix_versions = None
            if "fixVersions" in issue["fields"] and issue["fields"]["fixVersions"]:
                fix_versions = [version["name"] for version in issue["fields"]["fixVersions"]]
            
            all_issues.append({
                "Project": project_key,
                "Key": issue["key"],
                "Summary": issue["fields"]["summary"],
                "Status": issue["fields"]["status"]["name"],
                "Assignee": issue["fields"]["assignee"]["displayName"] if issue["fields"]["assignee"] else None,
                "Created": issue["fields"]["created"],
                "Reporter": issue["fields"]["reporter"]["displayName"] if issue["fields"]["reporter"] else None,
                "Resolution": issue["fields"]["resolution"]["name"] if issue["fields"]["resolution"] else None,
                "Priority": issue["fields"]["priority"]["name"] if issue["fields"]["priority"] else None,
                "Updated": issue["fields"]["updated"],
                "Type": issue["fields"]["issuetype"]["name"],
                "Affects Version": affects_version,
                "Components": [component["name"] for component in issue["fields"]["components"]],
                "Fix Versions": fix_versions,
                "Status Category": issue["fields"]["status"]["statusCategory"]["name"],
                "Description": description,
            })

        start_at += page_size
        if start_at >= data["total"]:
            break

    return all_issues

if __name__ == "__main__":
    username = "omar.khabthani123@gmail.com"
    api_token = APIKEY
    
    all_project_keys = fetch_all_project_keys()
    all_issues = []

    for project_key in all_project_keys:
        project_issues = fetch_issue_data(project_key)
        all_issues.extend(project_issues)
     
    df = pd.DataFrame(all_issues)
    excel_filename = "jira_fields1.xlsx"
    df.to_excel(excel_filename, index=False)
    print(f"DataFrame exported to {excel_filename} successfully.")

DataFrame exported to jira_fields1.xlsx successfully.
